In [1]:
import gurobipy as gb
import pandas as pd
import numpy as np
import random
from gurobipy import GRB

from utils import data_reader as dr
from utils import problem_model as pm


# Extra

In this part we try to add some further considerations about the models studied so far

## About agents' symmetry

In section 3.3. the authors of the paper said:
>    _"Of note, the increased convergence time may be a result of not only the increase in problem size,
>    but added symmetry within the problem between the homogeneous agents with identical source and termini nodes."_

In this part we investigate if lowering the symettry between agents' source and termini will speed-up the problem solutions. For simplicity we'll consider only the MSPP-PD-ABP problem

In [2]:
networks_df = dr.read_networks_csv("data/d_it_ij_6x6_1000it.csv",
                                   along="rows")

networks_df.head()


0                   1                             2             \
           6         7         6         7         8         7         8    
it1  1.612014  1.454776  0.376347  1.263488  1.675461  0.383242  1.325470   
it2  0.345666  1.997958  1.022579  1.344409  0.335500  0.768256  0.724642   
it3  1.059043  1.866680  1.672658  0.410748  1.204280  1.351971  1.502285   
it4  0.297746  1.873162  1.144040  1.749892  0.975469  0.168817  0.553882   
it5  1.296527  1.255480  1.777435  1.812928  1.990680  0.990032  0.467010   

                     3             ...        26                  27  \
           9         8         9   ...        32        33        32   
it1  1.502733  0.959483  1.690435  ...  0.199034  1.911133  1.343548   
it2  1.146472  0.726154  0.654525  ...  0.126858  0.794601  1.432874   
it3  0.093011  0.149816  0.665625  ...  1.773551  0.360475  0.016188   
it4  1.758605  1.824679  0.606143  ...  1.398726  1.794652  0.178435   
it5  0.629513  0.962954  0.695574  ...  0.499949  0.563903  0.976726   

                               28                            29            
           33        34        33        34        35        34        35  
it1  0.238743  1.667782  1.273160  0.289610  1.523664  1.968861  1.483620  
it2  0.123810  1.866517  1.411055  1.125656  1.531357  0.725753  0.617501  
it3  1.866432  1.783597  1.205805  0.093973  1.818958  1.471756  1.272506  
it4  0.930916  0.497418  1.837166  1.315127  0.259269  1.973621  1.269510  
it5  1.407947  0.320083  0.168943  0.432240  0.193984  1.377742  0.847393  

[5 rows x 80 columns]

Define data

In [3]:
network_shape = (6, 6)
network_nodes = pm.get_nodes(networks_df)
num_instances = 5

symmetry_levels = ["high", "medium", "low"]
idx_of_symmetry_lvl = dict(zip(symmetry_levels, range(len(symmetry_levels))))

scenarios = [3, 6, 9, 12]
idx_of_scenario = dict(zip(scenarios, range(len(scenarios))))

convergence_time_shape = (num_instances,
                          len(symmetry_levels),
                          len(scenarios))
convergence_times = np.zeros(convergence_time_shape)


Fix seed of the random number generator, that will be used to generate random agents' sources and terminus

In [4]:
random.seed(0)


In [ ]:
for it_i, network_arcs in enumerate(pm.network_instances(networks_df.head(num_instances))):

    for num_of_agents in scenarios:

        for symmetry_lvl in symmetry_levels:

            agents = pm.generate_agents(network_shape,
                                        num_of_agents,
                                        symmetry=symmetry_lvl)

            pb, X, *_ = pm.set_problem("ABP",
                                       network_nodes, network_arcs, agents)
            pb.optimize()

            if pb.Status == GRB.OPTIMAL:
                convergence_times[it_i,
                                  idx_of_symmetry_lvl[symmetry_lvl],
                                  idx_of_scenario[num_of_agents]] = pb.Runtime
            else:
                print(f"Something bad occurred for {symmetry_lvl=} in {it_i=}")


If desired we can store results in order to avoid waiting for computations each time we run the notebook

In [6]:
np.save(
    f"results/agent_symmetries_convergence_times_{num_instances}.npy", convergence_times)


In [ ]:
convergence_times = np.load(
    f"results/agent_symmetries_convergence_times_{num_instances}.npy")


In [11]:
avg_convergence_times = np.mean(convergence_times, axis=0)

avg_convergence_times_df = pd.DataFrame(avg_convergence_times,
                                        index=symmetry_levels,
                                        columns=scenarios)

avg_convergence_times_df

,3,6,9,12
high,0.012102,0.012497,0.011281,0.010626
medium,0.004124,0.008952,0.240642,0.573463
low,0.007431,0.059653,0.120813,0.116003


Interestingly, if all agents have the same source and terminus nodes, the MSPP-PD(ABP) require more or less the same time to be solved independently on the number of agents that has to be routed.

About the relationship between mean convergence time and symmetry of the problem we can state:

- If all agents start from the same source node and end in the same terminus node, the problem size does not change the mean convergence time;

- If all agent source nodes and terminus nodes are selected randomly from the first and last network columns respectively, the mean convergence time stop increasing after a certain problem size;

- If source node and terminus node of agents follows a more sophisticated pattern as the one described in the paper then the mean convergemce time increase with the problem size.


## A possible real case application